In [1]:
import os
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy

from sklearn import preprocessing, pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from cfmining.mip_algorithms import LinearRecourseActions, LinearRecourseActionsMulti
from cfmining.algorithms import MAPOCAM, BruteForce, Greedy
from cfmining.criteria import PercentileCalculator, PercentileCriterion, PercentileChangesCriterion, NonDomCriterion
from cfmining.predictors import MonotoneClassifier, GeneralClassifier, TreeClassifier, LinearClassifier
from cfmining.action_set import ActionSet

from results import save_result

Using Python-MIP package version 1.7.2


In [2]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

## Preparing dataset

In [3]:
giveme_df = pd.read_csv('data/givemecredit/cs-training.csv', index_col=0).dropna()
y = giveme_df['SeriousDlqin2yrs']
X = giveme_df.drop('SeriousDlqin2yrs', axis=1)

In [4]:
Xtr, Xts, ytr, yts = train_test_split(X, y, test_size=100)

## Setting ActionSet base parameters

In [5]:
action_set_base = ActionSet(X = X)
for feat in action_set_base:
    if feat.name in ['age', 'MonthlyIncome']:
        feat.mutable = False
        #feat.flip_direction = 1
        feat.step_direction = -1
    if feat.name in ['NumberOfDependents']:
        feat.mutable = False
        #feat.flip_direction = 1
        feat.step_direction = 1
    if feat.name in ['RevolvingUtilizationOfUnsecuredLines', 'DebtRatio']:
        feat.mutable = True
        #feat.flip_direction = -1
        feat.step_direction = -1
    if feat.name in ['NumberOfTime30-59DaysPastDueNotWorse',
                     'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
                     'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse']:
        feat.mutable = True
        #feat.flip_direction = -1
        feat.step_direction = 1
    #print(feat.name, feat.step_direction)
    feat.update_grid()

## Logistic regression model

In [6]:
stand = preprocessing.StandardScaler()
clf_base = LogisticRegression(max_iter=1000, class_weight='balanced')
clf = pipeline.Pipeline([('std', stand), ('clf', clf_base)])

grid = GridSearchCV(
  clf, param_grid={'clf__C': np.logspace(-4, 3)},
  cv=5,
  scoring='roc_auc',
  verbose=0
)

grid.fit(Xtr, ytr)
clf_lgr = grid.best_estimator_

In [7]:
threshold=0.5
scores = pd.Series(clf_lgr.predict_proba(Xts)[:, 1])
denied_individuals = scores.loc[lambda s: s < threshold].index

In [8]:
clf_lgr_mapocam = MonotoneClassifier(clf_lgr, X=Xtr, y=ytr, threshold=threshold)
coefficients = clf_lgr['clf'].coef_[0]/clf_lgr['std'].scale_
intercept = clf_lgr['clf'].intercept_[0]-coefficients@clf_lgr['std'].mean_

/home/marcos/Documents/Experiments/envs/mapocam/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/marcos/Documents/Experiments/envs/mapocam/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


## Preparing counterfactual parameters

In [9]:
action_set = copy.deepcopy(action_set_base)

print('ActionSet stats')
print('Number of actionable features:', sum([action.actionable for action in action_set]))
print('Mean number of actions per feature:', np.nanmean([len(action._grid) if action.actionable else np.nan for action in action_set]))
print('Max number of actions per feature:', np.nanmax([len(action._grid) if action.actionable else np.nan for action in action_set]))

ActionSet stats
Number of actionable features: 7
Mean number of actions per feature: 34.857142857142854
Max number of actions per feature: 101.0


In [10]:
percCalc = PercentileCalculator(action_set=action_set)

## Linear programming

In [11]:
lp_lgr_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = LinearRecourseActions(action_set, individual, coefficients, intercept, threshold)
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_lgr_results[i] = {}
    lp_lgr_results[i]['enum'] = lp
    lp_lgr_results[i]['indiv'] = individual
    lp_lgr_results[i]['solution'] = [int(s) if act.variable_type=='int' else float(s) for s, act in zip(lp.solution, action_set)]
    lp_lgr_results[i]['stats'] = lp.stats
    lp_lgr_results[i]['recourse'] = lp.recourse
    lp_lgr_results[i]['time'] = lp_time
    lp_lgr_results[i]['cost'] = criteria.f(lp_lgr_results[i]['solution']) if ~any(np.isnan(lp_lgr_results[i]['solution'])) else float('inf')

100%|██████████| 78/78 [00:02<00:00, 30.88it/s]


## Greedy approach

In [12]:
greedy_lgr_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    greedy = Greedy(action_set, individual, clf_lgr_mapocam, criteria)
    greedy.fit()
    greedy_time = time.perf_counter()-start
    
    greedy_lgr_results[i] = {}
    greedy_lgr_results[i]['solution'] = greedy.solution
    greedy_lgr_results[i]['time'] = greedy_time
    greedy_lgr_results[i]['cost'] = criteria.f(greedy.solution) if greedy.solution is not None else float('inf')

100%|██████████| 78/78 [00:01<00:00, 60.61it/s]


## MAPOCAM

In [13]:
mapocam_lgr_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam = MAPOCAM(action_set, individual, clf_lgr_mapocam, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_lgr_results[i] = {}
    mapocam_lgr_results[i]['enum'] = mapocam
    mapocam_lgr_results[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_lgr_results[i]['time'] = mapocam_time
    mapocam_lgr_results[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

100%|██████████| 78/78 [00:00<00:00, 226.61it/s]


## Performance comparison for single objective


In [14]:
lp_lgr_costs = np.array([lp_lgr_results[i]['cost'] for i in denied_individuals])
greedy_lgr_costs = np.array([greedy_lgr_results[i]['cost'] for i in denied_individuals])
mapocam_lgr_costs = np.array([mapocam_lgr_results[i]['cost'] for i in denied_individuals])

lp_lgr_time = np.array([lp_lgr_results[i]['time'] for i in denied_individuals])
greedy_lgr_time = np.array([greedy_lgr_results[i]['time'] for i in denied_individuals])
mapocam_lgr_time = np.array([mapocam_lgr_results[i]['time'] for i in denied_individuals])

best_costs = np.min([lp_lgr_costs, greedy_lgr_costs, mapocam_lgr_costs], axis=0)

In [15]:
lgr_dict = {'Time performance':
            {
                'MAPOCAM': float(np.mean(mapocam_lgr_time)),
                'greedy': float(np.mean(greedy_lgr_time)),
                'mi-logistic': float(np.mean(lp_lgr_time)),
            },
            'Best solution rate':
           {
               'MAPOCAM':float(np.mean(mapocam_lgr_costs==best_costs)),
               'greedy': float(np.mean(greedy_lgr_costs==best_costs)),
               'mi-logistic': float(np.mean(lp_lgr_costs==best_costs)),
           },
            'Feasible rate':1-float(np.mean(np.isnan(mapocam_lgr_costs)))
    
}
save_result(lgr_dict, 'giveme', 'logistic', 'MPC cost')

Best solution rate:
  MAPOCAM: 1.0
  greedy: 0.16666666666666666
  mi-logistic: 1.0
Feasible rate: 1.0
Time performance:
  MAPOCAM: 0.0042059649907362955
  greedy: 0.01619346625655173
  mi-logistic: 0.017176882581761442



## Comparing Pareto enumeration

## Percentile vs changes

In [16]:
mapocam_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam = MAPOCAM(action_set, individual,
                      clf_lgr_mapocam, max_changes=3,
                      compare=PercentileChangesCriterion(individual, percCalc),
                      recursive=False)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_results_pc[i] = {}
    mapocam_results_pc[i]['enum'] = mapocam
    mapocam_results_pc[i]['time'] = mapocam_time
    mapocam_results_pc[i]['num'] = len(mapocam.solutions)

100%|██████████| 78/78 [00:00<00:00, 157.33it/s]


In [17]:
lp_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = LinearRecourseActionsMulti(action_set, individual, coefficients, intercept,
                                    threshold, max_changes=3, enumeration_type='remove_number_actions',
                                    compare=PercentileChangesCriterion(individual, percCalc))
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_results_pc[i] = {}
    lp_results_pc[i]['enum'] = lp
    lp_results_pc[i]['time'] = lp_time
    lp_results_pc[i]['num'] = len(lp.solutions)

100%|██████████| 78/78 [00:01<00:00, 70.09it/s]


In [18]:
bf_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    bf = BruteForce(action_set, individual, clf_lgr_mapocam, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc))
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_results_pc[i] = {}
    bf_results_pc[i]['enum'] = bf
    bf_results_pc[i]['time'] = bf_time
    bf_results_pc[i]['num'] = len(bf.solutions)

100%|██████████| 78/78 [03:22<00:00,  2.60s/it]


In [19]:
brute_sols_pc = np.array([bf_results_pc[i]['num'] for i in denied_individuals])
mapocam_sols_pc = np.array([mapocam_results_pc[i]['num'] for i in denied_individuals])
lp_sols_pc = np.array([lp_results_pc[i]['num'] for i in denied_individuals])
best_sols_pc = brute_sols_pc

brute_time_pc = np.array([bf_results_pc[i]['time'] for i in denied_individuals])
mapocam_time_pc = np.array([mapocam_results_pc[i]['time'] for i in denied_individuals])
lp_time_pc = np.array([lp_results_pc[i]['time'] for i in denied_individuals])

In [20]:
lgr_dict_pc = {'Time performance':
            {
                'MAPOCAM': float(np.mean(mapocam_time_pc)),
                'brute-force': float(np.mean(brute_time_pc)),
                'po-mi-logistic': float(np.mean(lp_time_pc)),
            },
            'Best solution rate':
           {
               'MAPOCAM':float(np.mean(mapocam_sols_pc==best_sols_pc)),
               'brute-force': float(np.mean(brute_sols_pc==best_sols_pc)),
               'po-mi-logistic': float(np.mean(lp_sols_pc==best_sols_pc)),
           }
    
}
save_result(lgr_dict_pc, 'giveme', 'logistic', 'MPC vs. #changes')

Best solution rate:
  MAPOCAM: 1.0
  brute-force: 1.0
  po-mi-logistic: 1.0
Time performance:
  MAPOCAM: 0.0061621565344886714
  brute-force: 2.5937936801929027
  po-mi-logistic: 0.01394238698338039



## Features

In [21]:
mapocam_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam = MAPOCAM(action_set, individual, clf_lgr_mapocam, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_results_feat[i] = {}
    mapocam_results_feat[i]['enum'] = mapocam
    mapocam_results_feat[i]['time'] = mapocam_time
    mapocam_results_feat[i]['num'] = len(mapocam.solutions)

100%|██████████| 78/78 [00:01<00:00, 63.50it/s]


In [22]:
lp_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = LinearRecourseActionsMulti(action_set, individual, coefficients, intercept,
                                    threshold, max_changes=3, enumeration_type='remove_dominated')
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_results_feat[i] = {}
    lp_results_feat[i]['enum'] = lp
    lp_results_feat[i]['time'] = lp_time
    lp_results_feat[i]['num'] = len(lp.solutions)

100%|██████████| 78/78 [00:04<00:00, 17.58it/s]


In [23]:
bf_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    bf = BruteForce(action_set, individual, clf_lgr_mapocam, max_changes=3)
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_results_feat[i] = {}
    bf_results_feat[i]['enum'] = bf
    bf_results_feat[i]['time'] = bf_time
    bf_results_feat[i]['num'] = len(bf.solutions)

100%|██████████| 78/78 [02:21<00:00,  1.82s/it]


In [24]:
brute_sols_feat = np.array([bf_results_feat[i]['num'] for i in denied_individuals])
mapocam_sols_feat = np.array([mapocam_results_feat[i]['num'] for i in denied_individuals])
lp_sols_feat = np.array([lp_results_feat[i]['num'] for i in denied_individuals])
best_sols_feat = brute_sols_feat

brute_time_feat = np.array([bf_results_feat[i]['time'] for i in denied_individuals])
mapocam_time_feat = np.array([mapocam_results_feat[i]['time'] for i in denied_individuals])
lp_time_feat = np.array([lp_results_feat[i]['time'] for i in denied_individuals])

In [25]:
lgr_dict_feat = {'Time performance':
            {
                'MAPOCAM': float(np.mean(mapocam_time_feat)),
                'brute-force': float(np.mean(brute_time_feat)),
                'po-mi-logistic': float(np.mean(lp_time_feat)),
            },
            'Best solution rate':
           {
               'MAPOCAM':float(np.mean(mapocam_sols_feat==best_sols_feat)),
               'brute-force': float(np.mean(brute_sols_feat==best_sols_feat)),
               'po-mi-logistic': float(np.mean(lp_sols_feat==best_sols_feat)),
           }
    
}
save_result(lgr_dict_feat, 'giveme', 'logistic', 'feature')

Best solution rate:
  MAPOCAM: 1.0
  brute-force: 1.0
  po-mi-logistic: 1.0
Time performance:
  MAPOCAM: 0.015427698546913095
  brute-force: 1.814669221227702
  po-mi-logistic: 0.05590676398237403



## Random forest

In [78]:
from sklearn import tree, ensemble
from cfmining.mip_algorithms import ForestRecourseActions
from cfmining.predictors import TreeClassifier

In [79]:
clf_rt = ensemble.RandomForestClassifier(n_estimators=5, max_depth=5, max_leaf_nodes=31,
                                         class_weight='balanced_subsample')
clf_rt.fit(Xtr, ytr);

## Preparing counterfactual parameters

In [92]:
action_set_tree = copy.deepcopy(action_set_base)
action_set_tree.embed_forest(clf_rt)
for action in action_set_tree:
    action.flip_direction = action.step_direction
    
print('ActionSet stats')
print('Number of actionable features:', sum([action.mutable for action in action_set_tree]))
print('Mean number of actions per feature:', np.mean([len(action._grid)-1 for action in action_set_tree]))
print('Max number of actions per feature:', np.max([len(action._grid)-1 for action in action_set_tree]))

ActionSet stats
Number of actionable features: 7
Mean number of actions per feature: 8.5
Max number of actions per feature: 27


In [93]:
scores = pd.Series(clf_rt.predict_proba(Xts)[:, 1])
denied_individuals = scores.loc[lambda s: s < threshold].index

In [94]:
percCalc = PercentileCalculator(action_set=action_set_tree)

In [95]:
mapocam_clf_rf = TreeClassifier(clf_rt, Xtr, ytr, threshold=threshold)
mapocam_clf_rf_fast = TreeClassifier(clf_rt, Xtr, ytr, threshold=threshold, use_predict_max=True)

## Linear programming

In [96]:
lp_rf_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_tree, individual, clf_rt, threshold=threshold, max_changes=float('inf'))
    lp.build_model()
    lp.fit(threads=1)
    lp_time = time.perf_counter()-start
    
    lp_rf_results[i] = {}
    lp_rf_results[i]['enum'] = lp
    lp_rf_results[i]['indiv'] = individual
    lp_rf_results[i]['solution'] = lp.solution#[int(s) if act.variable_type=='int' else float(s) for s, act in zip(pl.solution, action_set)]
    lp_rf_results[i]['time'] = lp_time
    lp_rf_results[i]['cost'] = criteria.f(lp_rf_results[i]['solution']) if not any(np.isnan(lp_rf_results[i]['solution'])) else float('inf')

100%|██████████| 77/77 [00:01<00:00, 48.93it/s]


## Greedy approach

In [97]:
greedy_rf_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    greedy = Greedy(action_set_tree, individual, mapocam_clf_rf_fast, criteria)
    greedy.fit()
    greedy_time = time.perf_counter()-start
    
    greedy_rf_results[i] = {}
    greedy_rf_results[i]['solution'] = greedy.solution
    greedy_rf_results[i]['time'] = greedy_time
    greedy_rf_results[i]['cost'] = criteria.f(greedy.solution) if greedy.solution is not None else float('inf')

100%|██████████| 77/77 [00:00<00:00, 140.53it/s]


## MAPOCAM

In [98]:
mapocam_rf_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_rf.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual, mapocam_clf_rf, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results[i] = {}
    mapocam_rf_results[i]['enum'] = mapocam
    mapocam_rf_results[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_rf_results[i]['time'] = mapocam_time
    mapocam_rf_results[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

100%|██████████| 77/77 [00:00<00:00, 123.11it/s]


In [99]:
mapocam_rf_results_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual, mapocam_clf_rf_fast, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_plus[i] = {}
    mapocam_rf_results_plus[i]['enum'] = mapocam
    mapocam_rf_results_plus[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_rf_results_plus[i]['time'] = mapocam_time
    mapocam_rf_results_plus[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

100%|██████████| 77/77 [00:00<00:00, 127.21it/s]


In [100]:
lp_rf_costs = np.array([lp_rf_results[i]['cost'] for i in denied_individuals])
greedy_rf_costs = np.array([greedy_rf_results[i]['cost'] for i in denied_individuals])
mapocam_rf_costs = np.array([mapocam_rf_results[i]['cost'] for i in denied_individuals])
mapocam_rf_costs_plus = np.array([mapocam_rf_results_plus[i]['cost'] for i in denied_individuals])
best_costs_rf = np.min([lp_rf_costs, greedy_rf_costs, mapocam_rf_costs, mapocam_rf_costs_plus], axis=0)

lp_rf_time = np.array([lp_rf_results[i]['time'] for i in denied_individuals])
greedy_rf_time = np.array([greedy_rf_results[i]['time'] for i in denied_individuals])
mapocam_rf_time = np.array([mapocam_rf_results[i]['time'] for i in denied_individuals])
mapocam_rf_time_plus = np.array([mapocam_rf_results_plus[i]['time'] for i in denied_individuals])

In [101]:
lp_rf_costs

array([0.00189509, 0.00189509, 0.04633685, 0.04633685, 0.04633685,
       0.04633685, 0.04633685, 0.04633685, 0.04995867, 0.04633685,
       0.04633685, 0.04633685, 0.04633685, 0.04633685, 0.04633685,
       0.04995867, 0.04633685, 0.04995867, 0.04633685, 0.00189509,
       0.04633685, 0.04633685, 0.04633685, 0.04633685, 0.04633685,
       0.04633685, 0.04995867, 0.04633685, 0.04633685, 0.04633685,
       0.00189509, 0.04633685, 0.04633685, 0.04633685, 0.04633685,
       0.04633685, 0.04633685, 0.04633685, 0.04633685, 0.04633685,
       0.04633685, 0.04633685, 0.04633685, 0.00189509, 0.04633685,
       0.04633685, 0.04633685, 0.04633685, 0.04633685, 0.04633685,
       0.04633685, 0.04633685, 0.04633685, 0.02361717, 0.00189509,
       0.04995867, 0.04633685, 0.04633685, 0.04633685, 0.04633685,
       0.04995867, 0.04633685, 0.04995867, 0.04633685, 0.04633685,
       0.04995867, 0.04633685, 0.04633685, 0.04995867, 0.04633685,
       0.04995867, 0.04633685, 0.04633685, 0.04633685, 0.04633

In [102]:
mapocam_rf_costs

array([0.        , 0.00189509, 0.04633685, 0.04633685, 0.04633685,
       0.04633685, 0.04633685, 0.04633685, 0.04995867, 0.04633685,
       0.04633685, 0.04633685, 0.04633685, 0.04633685, 0.04633685,
       0.04633685, 0.04633685, 0.04995867, 0.04633685, 0.00189509,
       0.04633685, 0.04633685, 0.04633685, 0.04633685, 0.04633685,
       0.04633685, 0.04995867, 0.04633685, 0.04633685, 0.04633685,
       0.        , 0.00189509, 0.04633685, 0.04633685, 0.04633685,
       0.00189509, 0.04633685, 0.04633685, 0.04633685, 0.04633685,
       0.04633685, 0.04633685, 0.        , 0.        , 0.04633685,
       0.04633685, 0.04633685, 0.04633685, 0.04633685, 0.04633685,
       0.04633685, 0.04633685, 0.04633685, 0.02361717, 0.00189509,
       0.04633685, 0.04633685, 0.04633685, 0.04633685, 0.04633685,
       0.04995867, 0.04633685, 0.04995867, 0.04633685, 0.04633685,
       0.04995867, 0.04633685, 0.04633685, 0.04995867, 0.04633685,
       0.04995867, 0.04633685, 0.04633685, 0.04633685, 0.04633

In [103]:
tree_dict = {'Time performance':
            {
                'MAPOCAM-P': float(np.mean(mapocam_rf_time_plus)),
                'MAPOCAM-N': float(np.mean(mapocam_rf_time)),
                'greedy': float(np.mean(greedy_rf_time)),
                'mi-trees': float(np.mean(lp_rf_time)),
            },
            'Best solution rate':
           {
               'MAPOCAM-P':float(np.mean(mapocam_rf_costs_plus==best_costs_rf)),
               'MAPOCAM-N':float(np.mean(mapocam_rf_costs==best_costs_rf)),
               'greedy': float(np.mean(greedy_rf_costs==best_costs_rf)),
               'mi-trees': float(np.mean(lp_rf_costs==best_costs_rf)),
           },
             'Feasible rate':1-float(np.mean(np.isnan(mapocam_rf_costs_plus)))
    
}
save_result(tree_dict, 'giveme', 'tree', 'MPC cost')

Best solution rate:
  MAPOCAM-N: 1.0
  MAPOCAM-P: 1.0
  greedy: 0.8571428571428571
  mi-trees: 0.8961038961038961
Feasible rate: 1.0
Time performance:
  MAPOCAM-N: 0.007894436207406416
  MAPOCAM-P: 0.007644488749868394
  greedy: 0.0069071810689524974
  mi-trees: 0.014402866127670973



## Comparing Pareto enumeration

## Percentile vs changes

In [38]:
mapocam_rf_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual,
                      mapocam_clf_rf, max_changes=3,
                      compare=PercentileChangesCriterion(individual, percCalc),
                      recursive=False)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_pc[i] = {}
    mapocam_rf_results_pc[i]['enum'] = mapocam
    mapocam_rf_results_pc[i]['time'] = mapocam_time
    mapocam_rf_results_pc[i]['num'] = len(mapocam.solutions)

100%|██████████| 88/88 [00:02<00:00, 35.96it/s]


In [39]:
mapocam_rf_results_pc_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual,
                      mapocam_clf_rf_fast, max_changes=3,
                      compare=PercentileChangesCriterion(individual, percCalc),
                      recursive=False)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_pc_plus[i] = {}
    mapocam_rf_results_pc_plus[i]['enum'] = mapocam
    mapocam_rf_results_pc_plus[i]['time'] = mapocam_time
    mapocam_rf_results_pc_plus[i]['num'] = len(mapocam.solutions)

100%|██████████| 88/88 [00:02<00:00, 42.17it/s]


In [40]:
lp_rf_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_tree, individual, clf_rt, threshold=threshold,
                               max_changes=3, multi_solution=True,
                               cost_type='linear', compare=PercentileChangesCriterion(individual, percCalc))
    lp.build_model()
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_rf_results_pc[i] = {}
    lp_rf_results_pc[i]['enum'] = lp
    lp_rf_results_pc[i]['time'] = lp_time
    lp_rf_results_pc[i]['num'] = len(lp.solutions)

100%|██████████| 88/88 [00:04<00:00, 19.70it/s]


In [41]:
bf_rf_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    bf = BruteForce(action_set_tree, individual, mapocam_clf_rf_fast, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc))
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_rf_results_pc[i] = {}
    bf_rf_results_pc[i]['enum'] = bf
    bf_rf_results_pc[i]['time'] = bf_time
    bf_rf_results_pc[i]['num'] = len(bf.solutions)

100%|██████████| 88/88 [00:31<00:00,  2.81it/s]


In [42]:
lp_rf_sols_pc = np.array([lp_rf_results_pc[i]['num'] for i in denied_individuals])
brute_rf_sols_pc = np.array([bf_rf_results_pc[i]['num'] for i in denied_individuals])
mapocam_rf_sols_pc = np.array([mapocam_rf_results_pc[i]['num'] for i in denied_individuals])
mapocam_rf_sols_pc_plus = np.array([mapocam_rf_results_pc_plus[i]['num'] for i in denied_individuals])
best_rf_sols_pc = brute_rf_sols_pc

lp_rf_time_pc = np.array([lp_rf_results_pc[i]['time'] for i in denied_individuals])
brute_rf_time_pc = np.array([bf_rf_results_pc[i]['time'] for i in denied_individuals])
mapocam_rf_time_pc = np.array([mapocam_rf_results_pc[i]['time'] for i in denied_individuals])
mapocam_rf_time_pc_plus = np.array([mapocam_rf_results_pc_plus[i]['time'] for i in denied_individuals])

In [43]:
tree_dict_pc = {'Time performance':
            {
                'MAPOCAM-P': float(np.mean(mapocam_rf_time_pc_plus)),
                'MAPOCAM-N': float(np.mean(mapocam_rf_time_pc)),
                'brute-force': float(np.mean(brute_rf_time_pc)),
                'po-mi-trees': float(np.mean(lp_rf_time_pc)),
            },
            'Best solution rate':
           {
               'MAPOCAM-P':float(np.mean(mapocam_rf_sols_pc_plus==best_rf_sols_pc)),
               'MAPOCAM-N':float(np.mean(mapocam_rf_sols_pc==best_rf_sols_pc)),
               'brute-force': float(np.mean(brute_rf_sols_pc==best_rf_sols_pc)),
               'po-mi-trees': float(np.mean(lp_rf_sols_pc==best_rf_sols_pc)),
           }
    
}
save_result(tree_dict_pc, 'giveme', 'tree', 'MPC vs. #changes')

Best solution rate:
  MAPOCAM-N: 1.0
  MAPOCAM-P: 1.0
  brute-force: 1.0
  po-mi-trees: 1.0
Time performance:
  MAPOCAM-N: 0.02744091741095128
  MAPOCAM-P: 0.02336445981266231
  brute-force: 0.35410890529403166
  po-mi-trees: 0.05028210709993304



## Features

In [44]:
mapocam_rf_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual, mapocam_clf_rf, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_feat[i] = {}
    mapocam_rf_results_feat[i]['enum'] = mapocam
    mapocam_rf_results_feat[i]['time'] = mapocam_time
    mapocam_rf_results_feat[i]['num'] = len(mapocam.solutions)

100%|██████████| 88/88 [00:03<00:00, 26.35it/s]


In [45]:
mapocam_clf_rf_fast = TreeClassifier(clf_rt, Xtr, ytr, threshold=threshold, use_predict_max=True)

In [46]:
mapocam_rf_results_feat_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual, mapocam_clf_rf_fast, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_feat_plus[i] = {}
    mapocam_rf_results_feat_plus[i]['enum'] = mapocam
    mapocam_rf_results_feat_plus[i]['time'] = mapocam_time
    mapocam_rf_results_feat_plus[i]['num'] = len(mapocam.solutions)

100%|██████████| 88/88 [00:01<00:00, 60.66it/s]


In [47]:
lp_rf_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_tree, individual, clf_rt, threshold=threshold,
                               max_changes=3, multi_solution=True,
                               cost_type='linear')
    lp.build_model()
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_rf_results_feat[i] = {}
    lp_rf_results_feat[i]['enum'] = lp
    lp_rf_results_feat[i]['time'] = lp_time
    lp_rf_results_feat[i]['num'] = len(lp.solutions)

100%|██████████| 88/88 [00:02<00:00, 42.07it/s]


In [48]:
bf_rf_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    bf = BruteForce(action_set_tree, individual, mapocam_clf_rf_fast, max_changes=3)
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_rf_results_feat[i] = {}
    bf_rf_results_feat[i]['enum'] = bf
    bf_rf_results_feat[i]['time'] = bf_time
    bf_rf_results_feat[i]['num'] = len(bf.solutions)

100%|██████████| 88/88 [00:18<00:00,  4.74it/s]


In [49]:
brute_rf_sols_feat = np.array([bf_rf_results_feat[i]['num'] for i in denied_individuals])
lp_rf_sols_feat = np.array([lp_rf_results_feat[i]['num'] for i in denied_individuals])
mapocam_rf_sols_feat = np.array([mapocam_rf_results_feat[i]['num'] for i in denied_individuals])
mapocam_rf_sols_feat_plus = np.array([mapocam_rf_results_feat_plus[i]['num'] for i in denied_individuals])
best_rf_sols_feat = brute_rf_sols_feat

brute_rf_time_feat = np.array([bf_rf_results_feat[i]['time'] for i in denied_individuals])
lp_rf_time_feat = np.array([lp_rf_results_feat[i]['time'] for i in denied_individuals])
mapocam_rf_time_feat = np.array([mapocam_rf_results_feat[i]['time'] for i in denied_individuals])
mapocam_rf_time_feat_plus = np.array([mapocam_rf_results_feat_plus[i]['time'] for i in denied_individuals])

In [50]:
tree_dict_feat = {'Time performance':
            {
                'MAPOCAM-P': float(np.mean(mapocam_rf_time_feat_plus)),
                'MAPOCAM-N': float(np.mean(mapocam_rf_time_feat)),
                'brute-force': float(np.mean(brute_rf_time_feat)),
                'po-mi-trees': float(np.mean(lp_rf_time_feat)),
            },
            'Best solution rate':
           {
               'MAPOCAM-P':float(np.mean(mapocam_rf_sols_feat_plus==best_rf_sols_feat)),
               'MAPOCAM-N':float(np.mean(mapocam_rf_sols_feat==best_rf_sols_feat)),
               'brute-force': float(np.mean(brute_rf_sols_feat==best_rf_sols_feat)),
               'po-mi-trees': float(np.mean(lp_rf_sols_feat==best_rf_sols_feat)),
           }
    
}
save_result(tree_dict_feat, 'giveme', 'tree', 'feature')

Best solution rate:
  MAPOCAM-N: 1.0
  MAPOCAM-P: 0.9090909090909091
  brute-force: 1.0
  po-mi-trees: 0.625
Time performance:
  MAPOCAM-N: 0.037567015634108844
  MAPOCAM-P: 0.01621364860462067
  brute-force: 0.2096239771344699
  po-mi-trees: 0.023371194457550617



## Monotone trees

In [51]:
from sklearn import tree, ensemble
import lightgbm
from cfmining.predictors import MonotoneTree

In [52]:
monotone_constraints = [action.step_direction for action in action_set_base]
clf_mt = lightgbm.LGBMClassifier(n_estimators=10, max_depth=10, num_leaves=63, monotone_constraints=monotone_constraints,
                                 class_weight='balanced')
clf_mt.fit(Xtr, ytr)

LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
               colsample_bytree=1.0, importance_type='split', learning_rate=0.1,
               max_depth=10, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0,
               monotone_constraints=[-1, -1, 1, -1, -1, 1, 1, 1, 1, 1],
               n_estimators=10, n_jobs=-1, num_leaves=63, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

## Preparing counterfactual parameters

In [53]:
action_set_mtree = copy.deepcopy(action_set_base)
action_set_mtree.embed_forest(clf_mt)
for action in action_set_mtree:
    action.flip_direction = action.step_direction

print('ActionSet stats')
print('Number of actionable features:', sum([action.mutable for action in action_set_mtree]))
print('Mean number of actions per feature:', np.mean([len(action._grid)-1 for action in action_set_mtree]))
print('Max number of actions per feature:', np.max([len(action._grid)-1 for action in action_set_mtree]))

ActionSet stats
Number of actionable features: 7
Mean number of actions per feature: 16.3
Max number of actions per feature: 66


In [54]:
scores = pd.Series(clf_mt.predict_proba(Xts)[:, 1])
denied_individuals = scores.loc[lambda s: s < threshold].index

In [55]:
percCalc = PercentileCalculator(action_set=action_set_mtree)

In [56]:
mapocam_clf_mt = TreeClassifier(clf_mt, Xtr, ytr, clf_type='lightgbm', threshold=threshold)
mapocam_clf_mt_fast = TreeClassifier(clf_mt, Xtr, ytr, clf_type='lightgbm', threshold=threshold, use_predict_max=True)
mapocam_clf_mt_mon = MonotoneTree(clf_mt, Xtr, ytr, clf_type='lightgbm', threshold=threshold, use_predict_max=True)

## Linear programming

In [57]:
lp_mt_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_mtree, individual, clf_mt, clf_type='lightgbm', threshold=threshold, max_changes=float('inf'))
    lp.build_model()
    lp.fit(threads=1)
    lp_time = time.perf_counter()-start
    
    lp_mt_results[i] = {}
    lp_mt_results[i]['enum'] = lp
    lp_mt_results[i]['indiv'] = individual
    lp_mt_results[i]['solution'] = lp.solution#[int(s) if act.variable_type=='int' else float(s) for s, act in zip(pl.solution, action_set)]
    lp_mt_results[i]['time'] = lp_time
    lp_mt_results[i]['cost'] = criteria.f(lp_mt_results[i]['solution']) if not any(np.isnan(lp_mt_results[i]['solution'])) else float('inf')

100%|██████████| 77/77 [00:03<00:00, 24.15it/s]


## Greedy approach

In [58]:
greedy_mt_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_mt.fit(individual, action_set_mtree)
    greedy = Greedy(action_set_mtree, individual, mapocam_clf_mt, criteria)
    greedy.fit()
    greedy_time = time.perf_counter()-start
    
    greedy_mt_results[i] = {}
    greedy_mt_results[i]['solution'] = greedy.solution
    greedy_mt_results[i]['time'] = greedy_time
    greedy_mt_results[i]['cost'] = criteria.f(greedy.solution) if greedy.solution is not None else float('inf')

100%|██████████| 77/77 [00:01<00:00, 57.84it/s]


## MAPOCAME

In [59]:
mapocam_mt_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_mt.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results[i] = {}
    mapocam_mt_results[i]['enum'] = mapocam
    mapocam_mt_results[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_mt_results[i]['time'] = mapocam_time
    mapocam_mt_results[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

100%|██████████| 77/77 [00:01<00:00, 62.70it/s]


In [60]:
mapocam_mt_results_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_fast, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_plus[i] = {}
    mapocam_mt_results_plus[i]['enum'] = mapocam
    mapocam_mt_results_plus[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_mt_results_plus[i]['time'] = mapocam_time
    mapocam_mt_results_plus[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

100%|██████████| 77/77 [00:01<00:00, 75.83it/s]


In [61]:
mapocam_mt_results_mon = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_mt_mon.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_mon, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_mon[i] = {}
    mapocam_mt_results_mon[i]['enum'] = mapocam
    mapocam_mt_results_mon[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_mt_results_mon[i]['time'] = mapocam_time
    mapocam_mt_results_mon[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

100%|██████████| 77/77 [00:01<00:00, 75.96it/s]


In [62]:
lp_mt_costs = np.array([lp_mt_results[i]['cost'] for i in denied_individuals])
greedy_mt_costs = np.array([greedy_mt_results[i]['cost'] for i in denied_individuals])
mapocam_mt_costs = np.array([mapocam_mt_results[i]['cost'] for i in denied_individuals])
mapocam_mt_costs_plus = np.array([mapocam_mt_results_plus[i]['cost'] for i in denied_individuals])
mapocam_mt_costs_mon = np.array([mapocam_mt_results_mon[i]['cost'] for i in denied_individuals])
best_costs_mt_rf = np.min([lp_mt_costs, greedy_mt_costs, mapocam_mt_costs, mapocam_mt_costs_plus, mapocam_mt_costs_mon], axis=0)

lp_mt_time = np.array([lp_mt_results[i]['time'] for i in denied_individuals])
greedy_mt_time = np.array([greedy_mt_results[i]['time'] for i in denied_individuals])
mapocam_mt_time = np.array([mapocam_mt_results[i]['time'] for i in denied_individuals])
mapocam_mt_time_plus = np.array([mapocam_mt_results_plus[i]['time'] for i in denied_individuals])
mapocam_mt_time_mon = np.array([mapocam_mt_results_mon[i]['time'] for i in denied_individuals])

In [63]:
tree_dict = {'Time performance':
            {
                'MAPOCAM-M': float(np.mean(mapocam_mt_time_mon)),
                'MAPOCAM-P': float(np.mean(mapocam_mt_time_plus)),
                'MAPOCAM-N': float(np.mean(mapocam_mt_time)),
                'greedy': float(np.mean(greedy_mt_time)),
                'mi-trees': float(np.mean(lp_mt_time)),
            },
            'Best solution rate':
           {
               'MAPOCAM-M':float(np.mean(mapocam_mt_costs_mon==best_costs_mt_rf)),
               'MAPOCAM-P':float(np.mean(mapocam_mt_costs_plus==best_costs_mt_rf)),
               'MAPOCAM-N':float(np.mean(mapocam_mt_costs==best_costs_mt_rf)),
               'greedy': float(np.mean(greedy_mt_costs==best_costs_mt_rf)),
               'mi-trees': float(np.mean(lp_mt_costs==best_costs_mt_rf)),
           },
             'Feasible rate':1-float(np.mean(np.isnan(mapocam_mt_costs_mon)))
    
}
save_result(tree_dict, 'giveme', 'monotone-tree', 'MPC cost')

Best solution rate:
  MAPOCAM-M: 1.0
  MAPOCAM-N: 1.0
  MAPOCAM-P: 1.0
  greedy: 0.42857142857142855
  mi-trees: 1.0
Feasible rate: 1.0
Time performance:
  MAPOCAM-M: 0.012894895804634729
  MAPOCAM-N: 0.015675174002838018
  MAPOCAM-P: 0.012914909006023852
  greedy: 0.016978453297482488
  mi-trees: 0.032799489254906



## Comparing Pareto enumeration

## Percentile vs changes

In [64]:
mapocam_mt_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual,
                      mapocam_clf_mt, max_changes=3,
                      compare=PercentileChangesCriterion(individual, percCalc),
                      recursive=False)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_pc[i] = {}
    mapocam_mt_results_pc[i]['enum'] = mapocam
    mapocam_mt_results_pc[i]['time'] = mapocam_time
    mapocam_mt_results_pc[i]['num'] = len(mapocam.solutions)

100%|██████████| 77/77 [00:01<00:00, 69.23it/s]


In [65]:
mapocam_mt_results_pc_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual,
                      mapocam_clf_mt_fast, max_changes=3,
                      compare=PercentileChangesCriterion(individual, percCalc),
                      recursive=False)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_pc_plus[i] = {}
    mapocam_mt_results_pc_plus[i]['enum'] = mapocam
    mapocam_mt_results_pc_plus[i]['time'] = mapocam_time
    mapocam_mt_results_pc_plus[i]['num'] = len(mapocam.solutions)

100%|██████████| 77/77 [00:01<00:00, 72.82it/s]


In [66]:
mapocam_mt_results_pc_mon = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_mon.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_mon, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc), recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_pc_mon[i] = {}
    mapocam_mt_results_pc_mon[i]['enum'] = mapocam
    mapocam_mt_results_pc_mon[i]['time'] = mapocam_time
    mapocam_mt_results_pc_mon[i]['num'] = len(mapocam.solutions)

100%|██████████| 77/77 [00:01<00:00, 63.47it/s]


In [67]:
lp_mt_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_mtree, individual, clf_mt, clf_type='lightgbm', threshold=threshold,
                               max_changes=3, multi_solution=True,
                               cost_type='linear', compare=PercentileChangesCriterion(individual, percCalc))
    lp.build_model()
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_mt_results_pc[i] = {}
    lp_mt_results_pc[i]['enum'] = lp
    lp_mt_results_pc[i]['time'] = lp_time
    lp_mt_results_pc[i]['num'] = len(lp.solutions)

100%|██████████| 77/77 [00:03<00:00, 23.98it/s]


In [68]:
bf_mt_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    bf = BruteForce(action_set_mtree, individual, mapocam_clf_mt_fast, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc))
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_mt_results_pc[i] = {}
    bf_mt_results_pc[i]['enum'] = bf
    bf_mt_results_pc[i]['time'] = bf_time
    bf_mt_results_pc[i]['num'] = len(bf.solutions)

100%|██████████| 77/77 [00:50<00:00,  1.53it/s]


In [69]:
brute_mt_sols_pc = np.array([bf_mt_results_pc[i]['num'] for i in denied_individuals])
lp_mt_sols_pc = np.array([lp_mt_results_pc[i]['num'] for i in denied_individuals])
mapocam_mt_sols_pc = np.array([mapocam_mt_results_pc[i]['num'] for i in denied_individuals])
mapocam_mt_sols_pc_plus = np.array([mapocam_mt_results_pc_plus[i]['num'] for i in denied_individuals])
mapocam_mt_sols_pc_mon = np.array([mapocam_mt_results_pc_mon[i]['num'] for i in denied_individuals])
best_mt_sols_pc = brute_mt_sols_pc

brute_mt_time_pc = np.array([bf_mt_results_pc[i]['time'] for i in denied_individuals])
lp_mt_time_pc = np.array([lp_mt_results_pc[i]['time'] for i in denied_individuals])
mapocam_mt_time_pc = np.array([mapocam_mt_results_pc[i]['time'] for i in denied_individuals])
mapocam_mt_time_pc_plus = np.array([mapocam_mt_results_pc_plus[i]['time'] for i in denied_individuals])
mapocam_mt_time_pc_mon = np.array([mapocam_mt_results_pc_mon[i]['time'] for i in denied_individuals])

In [70]:
mon_tree_dict_pc = {'Time performance':
            {
                'MAPOCAM-M': float(np.mean(mapocam_mt_time_pc_mon)),
                'MAPOCAM-P': float(np.mean(mapocam_mt_time_pc_plus)),
                'MAPOCAM-N': float(np.mean(mapocam_mt_time_pc)),
                'brute-force': float(np.mean(brute_mt_time_pc)),
                'po-mi-trees': float(np.mean(lp_mt_time_pc)),
            },
            'Best solution rate':
           {
               'MAPOCAM-M':float(np.mean(mapocam_mt_sols_pc_plus==mapocam_mt_sols_pc_mon)),
               'MAPOCAM-P':float(np.mean(mapocam_mt_sols_pc_plus==best_mt_sols_pc)),
               'MAPOCAM-N':float(np.mean(mapocam_mt_sols_pc==best_mt_sols_pc)),
               'brute-force': float(np.mean(brute_mt_sols_pc==best_mt_sols_pc)),
               'po-mi-trees': float(np.mean(lp_mt_sols_pc==best_mt_sols_pc)),
           }
    
}
save_result(mon_tree_dict_pc, 'giveme', 'monotone-tree', 'MPC vs. #changes')

Best solution rate:
  MAPOCAM-M: 1.0
  MAPOCAM-N: 1.0
  MAPOCAM-P: 1.0
  brute-force: 1.0
  po-mi-trees: 1.0
Time performance:
  MAPOCAM-M: 0.015490269596845685
  MAPOCAM-N: 0.014179723896690971
  MAPOCAM-P: 0.013481665154701316
  brute-force: 0.6511621333631481
  po-mi-trees: 0.04113925493763784



## Features

In [71]:
mapocam_mt_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_feat[i] = {}
    mapocam_mt_results_feat[i]['enum'] = mapocam
    mapocam_mt_results_feat[i]['time'] = mapocam_time
    mapocam_mt_results_feat[i]['num'] = len(mapocam.solutions)

100%|██████████| 77/77 [00:07<00:00, 10.87it/s]


In [72]:
mapocam_mt_results_feat_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_fast, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_feat_plus[i] = {}
    mapocam_mt_results_feat_plus[i]['enum'] = mapocam
    mapocam_mt_results_feat_plus[i]['time'] = mapocam_time
    mapocam_mt_results_feat_plus[i]['num'] = len(mapocam.solutions)

100%|██████████| 77/77 [00:03<00:00, 19.82it/s]


In [73]:
mapocam_mt_results_feat_mon = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_mon.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_mon, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_feat_mon[i] = {}
    mapocam_mt_results_feat_mon[i]['enum'] = mapocam
    mapocam_mt_results_feat_mon[i]['time'] = mapocam_time
    mapocam_mt_results_feat_mon[i]['num'] = len(mapocam.solutions)

100%|██████████| 77/77 [00:01<00:00, 62.07it/s]


In [74]:
lp_mt_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_mtree, individual, clf_mt, clf_type='lightgbm', threshold=threshold,
                               max_changes=3, multi_solution=True,
                               cost_type='linear')
    lp.build_model()
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_mt_results_feat[i] = {}
    lp_mt_results_feat[i]['enum'] = lp
    lp_mt_results_feat[i]['time'] = lp_time
    lp_mt_results_feat[i]['num'] = len(lp.solutions)

100%|██████████| 77/77 [00:02<00:00, 26.05it/s]


In [75]:
bf_mt_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    bf = BruteForce(action_set_mtree, individual, mapocam_clf_mt_fast, max_changes=3)
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_mt_results_feat[i] = {}
    bf_mt_results_feat[i]['enum'] = bf
    bf_mt_results_feat[i]['time'] = bf_time
    bf_mt_results_feat[i]['num'] = len(bf.solutions)

100%|██████████| 77/77 [00:33<00:00,  2.30it/s]


In [76]:
brute_mt_sols_feat = np.array([bf_mt_results_feat[i]['num'] for i in denied_individuals])
lp_mt_sols_feat = np.array([lp_mt_results_feat[i]['num'] for i in denied_individuals])
mapocam_mt_sols_feat = np.array([mapocam_mt_results_feat[i]['num'] for i in denied_individuals])
mapocam_mt_sols_feat_plus = np.array([mapocam_mt_results_feat_plus[i]['num'] for i in denied_individuals])
mapocam_mt_sols_feat_mon = np.array([mapocam_mt_results_feat_mon[i]['num'] for i in denied_individuals])
best_mt_sols_feat = brute_mt_sols_feat

brute_mt_time_feat = np.array([bf_mt_results_feat[i]['time'] for i in denied_individuals])
lp_mt_time_feat = np.array([lp_mt_results_feat[i]['time'] for i in denied_individuals])
mapocam_mt_time_feat = np.array([mapocam_mt_results_feat[i]['time'] for i in denied_individuals])
mapocam_mt_time_feat_plus = np.array([mapocam_mt_results_feat_plus[i]['time'] for i in denied_individuals])
mapocam_mt_time_feat_mon = np.array([mapocam_mt_results_feat_mon[i]['time'] for i in denied_individuals])

In [77]:
mon_tree_dict_feat = {'Time performance':
            {
                'MAPOCAM-M': float(np.mean(mapocam_mt_time_feat_mon)),
                'MAPOCAM-P': float(np.mean(mapocam_mt_time_feat_plus)),
                'MAPOCAM-N': float(np.mean(mapocam_mt_time_feat)),
                'brute-force': float(np.mean(brute_mt_time_feat)),
                'po-mi-trees': float(np.mean(lp_mt_time_feat)),
            },
            'Best solution rate':
           {
               'MAPOCAM-M':float(np.mean(mapocam_mt_sols_feat_mon==best_mt_sols_feat)),
               'MAPOCAM-P':float(np.mean(mapocam_mt_sols_feat_plus==best_mt_sols_feat)),
               'MAPOCAM-N':float(np.mean(mapocam_mt_sols_feat==best_mt_sols_feat)),
               'brute-force': float(np.mean(brute_mt_sols_feat==best_mt_sols_feat)),
               'po-mi-trees': float(np.mean(lp_mt_sols_feat==best_mt_sols_feat)),
           }
    
}
save_result(mon_tree_dict_feat, 'giveme', 'monotone-tree', 'feature')

Best solution rate:
  MAPOCAM-M: 1.0
  MAPOCAM-N: 1.0
  MAPOCAM-P: 1.0
  brute-force: 1.0
  po-mi-trees: 0.6883116883116883
Time performance:
  MAPOCAM-M: 0.01585808554331639
  MAPOCAM-N: 0.09147005283588229
  MAPOCAM-P: 0.05008974346761628
  brute-force: 0.43394396591375206
  po-mi-trees: 0.037847967633929155

